In [1]:
# !pip3 install -U keras-tuner
# !pip3 install tensorflow
# !pip3 install imblearn

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras as k
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix
import json

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
dataset_url = 'https://www.openml.org/data/get_csv/4965303/flare.arff' 
dataset = np.genfromtxt(dataset_url, delimiter=',', skip_header=1)

x = dataset[:,:-4]
x_size = x.shape[1]
y = dataset[:,-1]

In [5]:
x.shape

(1066, 7)

### <font color="#CA3532">Definición del modelo</font>


In [6]:
# Definid el modelo con Keras

nn = Sequential()

### -------------------------------------------------------------------------------
### Añadir la capas completamente conectadas que consideréis al modelo
### -------------------------------------------------------------------------------
nn.add(Dense(12, activation="sigmoid"))
nn.add(Dense(12, activation="sigmoid"))
nn.add(Dense(1, activation="sigmoid"))

In [7]:
metrics = [
            keras.metrics.BinaryAccuracy(name='ACC'),
            keras.metrics.Precision(name='Prec'),
            keras.metrics.Recall(name='Rec'),
            keras.metrics.AUC(name='AUC'),
          ]

nn.compile(optimizer='SGD', loss="mse", metrics=metrics)
# nn.compile(optimizer='Adam', loss="binary_crossentropy", metrics=metrics)

### <font color="#CA3532">Conjuntos de entrenamiento y validación</font>

In [8]:
# Contrucción de los conjuntos de entrenamiento y validación

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2) 

### <font color="#CA3532">Visualización de resultados</font>

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

def show_metrics(history):
    for metric in history.history.keys():
        if not metric.startswith('val_'):
            plt.plot(history.history[metric], label=metric)
            plt.plot(history.history[f'val_{metric}'], label=f'val_{metric}')
            plt.title(metric)
            plt.ylabel('')
            plt.xlabel('Epoch')
            plt.legend(loc="upper left")
            plt.show()

### <font color="#CA3532">Entrenamiento de la red neuronal y evaluación</font>
Como podrás haber observado en problema no está balanceado (o está bastante desequilibrado), porque el número de ejemplos de cada clase es muy diferente.

In [10]:
# !pip3 install imbalanced-learn

In [11]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN
from collections import Counter
from sklearn.metrics import f1_score

#### RandomUnderSampler
Debido a al desequilibrio entre clases se aplica un random under sampler a partir de lo cual el numero de registros de cada clase se equilibra quitandose registros aleatorios de la clase mayoritaria quedando ambas clases con 145 registros. Debido a este equilibrio de clases se obtienen f1 scores equilibrados en entrenamiento para ambas clases ya que el modelo no pondera una clase debido a la equidad de registros de una clase respecto a la otra. Sin embargo en test generaliza peor para la clase 1, debido a que le faltan mas registros para el entrenamiento de esta clase para generalizar bien o quizas debido a sobre ajuste.

In [12]:
undersample = RandomUnderSampler()
X_under_train, y_under_train = undersample.fit_resample(x_train, y_train)
print(Counter(y_under_train))

Counter({0.0: 145, 1.0: 145})


In [13]:
epochs = 50
history = nn.fit(X_under_train, y_under_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_under_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_under_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))

print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.53      0.96      0.68       145
         1.0       0.79      0.16      0.26       145

    accuracy                           0.56       290
   macro avg       0.66      0.56      0.47       290
weighted avg       0.66      0.56      0.47       290

Test
              precision    recall  f1-score   support

         0.0       0.82      0.93      0.88       177
         1.0       0.14      0.05      0.08        37

    accuracy                           0.78       214
   macro avg       0.48      0.49      0.48       214
weighted avg       0.71      0.78      0.74       214

[[165  12]
 [ 35   2]]


#### EditedNearestNeighbours
Con esta técnicaa se eliminan registros haciendo uso de k-vecinos cercanos, a partir de la que se eliminan registros cercanos a la frontera de decisión para distinguir entre una clase y otra. Debido a que en la frontera de decisión no existen muchos registros de la clase 0 solo se eliminan unos pocos quedando las clases muy desequilibradas por lo que el modelo pondera la clase 0 por encima de la clase 1 quedando unos f1 scores muy altos para la clase 0 pero nulos para la clase 1.

In [14]:
enn = EditedNearestNeighbours()
X_under_train, y_under_train = enn.fit_resample(x_train, y_train)
print(Counter(y_under_train))

Counter({0.0: 485, 1.0: 145})


In [15]:
epochs = 50
history = nn.fit(X_under_train, y_under_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_under_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_under_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87       485
         1.0       0.00      0.00      0.00       145

    accuracy                           0.77       630
   macro avg       0.38      0.50      0.43       630
weighted avg       0.59      0.77      0.67       630

Test
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91       177
         1.0       0.00      0.00      0.00        37

    accuracy                           0.83       214
   macro avg       0.41      0.50      0.45       214
weighted avg       0.68      0.83      0.75       214

[[177   0]
 [ 37   0]]


#### TomekLinks
Haciendo uso de esta técnica se eliminan los registos de la clase numerosa haciendo uso de los enlaces de Tomek los cuales son enlaces entre clases proximas. Debido a que las clases no están muy próximas apenas se eliminan 5 registros de la clase 0 quedando las clases muy desequilibradas por lo que el modelo pondera la clase 0 por encima de la clase 1 quedando unos f1 scores muy altos para la clase 0 pero nulos para la clase 1.

In [16]:
tl = TomekLinks()
X_under_train, y_under_train = tl.fit_resample(x_train, y_train)
print(Counter(y_under_train))

Counter({0.0: 706, 1.0: 145})


In [17]:
epochs = 50
history = nn.fit(X_under_train, y_under_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_under_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_under_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91       706
         1.0       0.00      0.00      0.00       145

    accuracy                           0.83       851
   macro avg       0.41      0.50      0.45       851
weighted avg       0.69      0.83      0.75       851

Test
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91       177
         1.0       0.00      0.00      0.00        37

    accuracy                           0.83       214
   macro avg       0.41      0.50      0.45       214
weighted avg       0.68      0.83      0.75       214

[[177   0]
 [ 37   0]]


#### RandomOverSampler
Esta técnia para la clase minoritaria aumenta el número de registros hasta quedarse con el mismo número de registros que la clase mayoritaria. Los registros los aumenta escogiendo registros repetidos de la clase minoritaria de forma aleatoria. Debido a que el numero de registros de las clases se encuentra equilibrado obtiene buenos f1 scores para ambas clases en entrenamiento aunque algo peores en test como ocurria con Random Under Sampler debido a que aunque ahora tiene más registros de la clase 1 para entrenarse y puede ponderar de forma equilibrada ambas clases, no tiene suficientes registros de la clase 1 que le permita generalizar bien ya que es poco el conocimiento que adquiere de ellos ya que muchos están repetidos

In [18]:
oversample = RandomOverSampler()
X_over_train, y_over_train = oversample.fit_resample(x_train, y_train)
print(Counter(y_over_train))

Counter({0.0: 707, 1.0: 707})


In [19]:
epochs = 50
history = nn.fit(X_over_train, y_over_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_over_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_over_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.65      0.83      0.73       707
         1.0       0.76      0.55      0.64       707

    accuracy                           0.69      1414
   macro avg       0.71      0.69      0.68      1414
weighted avg       0.71      0.69      0.68      1414

Test
              precision    recall  f1-score   support

         0.0       0.89      0.84      0.86       177
         1.0       0.39      0.49      0.43        37

    accuracy                           0.78       214
   macro avg       0.64      0.66      0.65       214
weighted avg       0.80      0.78      0.79       214

[[149  28]
 [ 19  18]]


#### SMOTE
Esta técnicas generan muestras de la clase minoritaria a partir de la interpolación dejandolas equilibradas y consiguiendo buenos resultados en entrenamiento aunque nuevamente algo peores en test por lo que se ha explicado anteriormente.

In [20]:
sm = SMOTE()
X_over_train, y_over_train = sm.fit_resample(x_train, y_train)
print(Counter(y_over_train))

Counter({0.0: 707, 1.0: 707})


In [21]:
epochs = 50
history = nn.fit(X_over_train, y_over_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_over_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_over_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.65      0.76      0.70       707
         1.0       0.71      0.59      0.64       707

    accuracy                           0.67      1414
   macro avg       0.68      0.67      0.67      1414
weighted avg       0.68      0.67      0.67      1414

Test
              precision    recall  f1-score   support

         0.0       0.90      0.77      0.83       177
         1.0       0.35      0.59      0.44        37

    accuracy                           0.74       214
   macro avg       0.62      0.68      0.63       214
weighted avg       0.81      0.74      0.76       214

[[136  41]
 [ 15  22]]


#### ADASYN
Esta técnica genera muestras de la clase minoritaria a partir de la interpolación. A diferencia de Smote, Adasyn se centra en generar muestras, que se encuentran junto a las muestras originales que se clasifican erróneamente, utilizando una clasificación k-Vecinos más cercanos. Por su parte el algoritmo SMOTE no hace ninguna distinción entre muestras para ser clasificadas utilizando la regla de vecinos más cercanos.

Nuevamente obtiene resultados similares a Somte

In [22]:
ada = ADASYN()
X_over_train, y_over_train = ada.fit_resample(x_train, y_train)
print(Counter(y_over_train))

Counter({1.0: 733, 0.0: 707})


In [23]:
epochs = 50
history = nn.fit(X_over_train, y_over_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_over_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_over_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.62      0.68      0.65       707
         1.0       0.66      0.60      0.63       733

    accuracy                           0.64      1440
   macro avg       0.64      0.64      0.64      1440
weighted avg       0.64      0.64      0.64      1440

Test
              precision    recall  f1-score   support

         0.0       0.90      0.67      0.77       177
         1.0       0.29      0.65      0.40        37

    accuracy                           0.66       214
   macro avg       0.59      0.66      0.58       214
weighted avg       0.80      0.66      0.70       214

[[118  59]
 [ 13  24]]


### Conclusion
Las técnicas de Tome Links y edited nearest neighbours no se han comportado bien ya que no han eliminado suficientes registros de la clase mayoritaria ya que no cumplían muchas registros las condiciones de los dos algoritmos anteriores ponderando el modelo la clase 0 por encima de la 1 y obteniendose resultados malos para la clase 1. Sin embargo a partir de las otras técnicas se ha conseguido un equilibrio de registros entre clases por lo que el modelo no ha ponderado una clase sobre la otra consiguiendo buenos resultados en entrenamiento y, resultados decentes aunque peores en test debido a que haciendo uso de las técnicas de undersampling no se tenína suficientes registros de la clase 1 para modelas y entenar un modelo con gran capacidad de generalizacion en test y debido a que las técnicas de oversampling, los registros apenas han aportado nuevo conocimiento al modelo de la clase 1 por lo que tampoco ha sabido generalizar en test de la mejor forma

### Uso de class_weight pero con paquete que nos proporciona
Otra forma de conseguir un equilibrio de ponderación entre clases es computar pesos para cada clase, con resultados similares a las técnicas anteriores donde se conseguia un equilibrio de registros entre clases

In [24]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: class_weights[0], 1: class_weights[1]}
print(class_weights)

{0: 0.6025459688826026, 1: 2.9379310344827587}


In [25]:
epochs = 50

history = nn.fit(x_train, y_train, epochs=epochs, verbose=0, class_weight=class_weights, validation_data=(x_val, y_val))

y_pred_train = nn.predict(x_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.90      0.75      0.82       707
         1.0       0.33      0.59      0.42       145

    accuracy                           0.73       852
   macro avg       0.62      0.67      0.62       852
weighted avg       0.80      0.73      0.75       852

Test
              precision    recall  f1-score   support

         0.0       0.90      0.76      0.82       177
         1.0       0.34      0.59      0.43        37

    accuracy                           0.73       214
   macro avg       0.62      0.68      0.63       214
weighted avg       0.80      0.73      0.75       214

[[134  43]
 [ 15  22]]


### Conclusion
Nuevamente gracias a la ponderación se consiguen resultados decentes y que no pondere una clase sobre la otra pero la red tiene poca capacidad de generalizacion para la clase 1 debido a que le falta más conocimiento sobre esta para generalizar mejor

### Aplicar todas las ténicas a la vez.


#### RandomUnderSampler + RandomOverSampler
A partir de esta combiación de téncias se reducen registros de la clase mayoritaria la 0, eliminando registros de manera aleatoria y creamos registros para la clase minoritaria y se computan pesos para que no pondere la clase 0 la cual seguiría teniendo más registros que la clase 1, pero como se ha estado explicando la ponderación solo es uno de los problemas por lo que obtiene resultados decentes pero no muy precisos para la clase 1

In [26]:
print(Counter(y_train))

oversample = RandomOverSampler(sampling_strategy=0.4)
X_over_train, y_over_train = oversample.fit_resample(x_train, y_train)
print(Counter(y_over_train))

undersample = RandomUnderSampler(sampling_strategy=0.5)
X_ajust_train, y_ajust_train = undersample.fit_resample(X_over_train, y_over_train)
print(Counter(y_ajust_train))

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_ajust_train), y=y_ajust_train)
class_weights = {0: class_weights[0], 1: class_weights[1]}
print(class_weights)

Counter({0.0: 707, 1.0: 145})
Counter({0.0: 707, 1.0: 282})
Counter({0.0: 564, 1.0: 282})
{0: 0.75, 1: 1.5}


In [27]:
epochs = 50

history = nn.fit(X_ajust_train, y_ajust_train, epochs=epochs, verbose=0, class_weight=class_weights, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_ajust_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_ajust_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.79      0.75      0.77       564
         1.0       0.55      0.60      0.57       282

    accuracy                           0.70       846
   macro avg       0.67      0.68      0.67       846
weighted avg       0.71      0.70      0.70       846

Test
              precision    recall  f1-score   support

         0.0       0.90      0.75      0.82       177
         1.0       0.33      0.59      0.43        37

    accuracy                           0.72       214
   macro avg       0.62      0.67      0.62       214
weighted avg       0.80      0.72      0.75       214

[[133  44]
 [ 15  22]]


#### SMOTE + ENN
Estas técnicas se combinan con el fin de reducir el numero de muestras de la clase mayoritaria haciendo uso de EditedNearestNeighbours eliminando los registros de esta clase que se encuentran en la frontera de decisión, y conseguir más muestras de la clase minoritaria con SMOTE como a partir de interpolación consiguiendo cierto equilibrio entre las clases 0 y 1 obteniendo resultados similares al anterior caso

In [28]:
smoteen = SMOTEENN()
X_ajust_train, y_ajust_train = smoteen.fit_resample(x_train, y_train)
print(Counter(y_ajust_train))

Counter({0.0: 442, 1.0: 315})


In [29]:
epochs = 50

history = nn.fit(X_ajust_train, y_ajust_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_ajust_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_ajust_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.77      0.95      0.85       442
         1.0       0.89      0.61      0.73       315

    accuracy                           0.81       757
   macro avg       0.83      0.78      0.79       757
weighted avg       0.82      0.81      0.80       757

Test
              precision    recall  f1-score   support

         0.0       0.88      0.85      0.86       177
         1.0       0.37      0.43      0.40        37

    accuracy                           0.78       214
   macro avg       0.62      0.64      0.63       214
weighted avg       0.79      0.78      0.78       214

[[150  27]
 [ 21  16]]


#### SMOTE + Tomek
Estas técnicas se combinan con el fin de reducir el numero de muestras de la clase mayoritaria haciendo uso de Tomek eliminando los registros de esta clase que se encuentran con enlaces Tomek con un valor de distancia muy pequeño, y conseguir más muestras de la clase minoritaria con SMOTE como a partir de interpolación

In [30]:
smtomek = SMOTETomek()
X_ajust_train, y_ajust_train = smtomek.fit_resample(x_train, y_train)
print(Counter(y_ajust_train))

Counter({0.0: 706, 1.0: 706})


In [31]:
epochs = 50

history = nn.fit(X_ajust_train, y_ajust_train, epochs=epochs, verbose=0, validation_data=(x_val, y_val))

y_pred_train = nn.predict(X_ajust_train)
y_pred_train = y_pred_train > 0.5
y_pred = nn.predict(x_val)
y_pred = y_pred > 0.5

# show_metrics(history)
print("Train")
print(classification_report(y_ajust_train, y_pred_train, zero_division=0))
print("Test")
print(classification_report(y_val, y_pred, zero_division=0))
print(confusion_matrix(y_val, y_pred))

Train
              precision    recall  f1-score   support

         0.0       0.66      0.75      0.70       706
         1.0       0.71      0.62      0.66       706

    accuracy                           0.68      1412
   macro avg       0.69      0.68      0.68      1412
weighted avg       0.69      0.68      0.68      1412

Test
              precision    recall  f1-score   support

         0.0       0.90      0.74      0.81       177
         1.0       0.32      0.59      0.42        37

    accuracy                           0.71       214
   macro avg       0.61      0.67      0.62       214
weighted avg       0.80      0.71      0.74       214

[[131  46]
 [ 15  22]]


### Conclusion
La combinación de técnicas de reducción para la clase mayoritaria y ampliación de la clase minoritaria consiguen que no se pondere una clase sobre la otra y obtiene buenos resultados pero no termina de generalizar con precisión para la clase 1 debido a que faltan más registros para esta clase que aporten conocimiento a la red para distinguirlos de los de la otra clase.
